In [ ]:
!pip install geopy > delete.txt
!pip install datasets > delete.txt
!pip install torch torchvision datasets > delete.txt
!pip install huggingface_hub > delete.txt
!pip install pyhocon > delete.txt
!pip install transformers > delete.txt
!pip install gensim > delete.txt
!rm delete.txt

In [ ]:
!huggingface-cli login

In [1]:
from datasets import load_dataset

dataset_train = load_dataset("CISProject/FOX_NBC", split="train")

# MODIFY IT TO THE TEST DATASET PATH
dataset_test = load_dataset("CISProject/FOX_NBC", split="test")

In [4]:
import numpy as np
import torch
import re
from transformers import BertTokenizer
from transformers import RobertaTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_data(data,
                    mode="train",
                    vectorizer=None,
                    w2v_model=None,
                    max_features=4096,
                    max_seq_length=128,
                    num_proc=4):
    if w2v_model is None:
        raise ValueError("w2v_model must be provided for Word2Vec embeddings.")

    # tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    # 1. Clean text once
    def clean_text(examples):
        import re
        cleaned = []
        for text in examples["title"]:
            text = text.lower()
            text = re.sub(r'[^\w\s]', '', text)
            text = text.strip()
            cleaned.append(text)
        return {"clean_title": cleaned}

    data = data.map(clean_text, batched=True, num_proc=num_proc)

    # 2. Fit CountVectorizer on training data if needed
    if mode == "train" and vectorizer is None:
        # Collect all cleaned titles to fit
        all_titles = data["clean_title"]
        #vectorizer = CountVectorizer(max_features=max_features, ngram_range=(1,2))
        vectorizer = TfidfVectorizer(max_features=max_features)
        vectorizer.fit(all_titles)
        print("vectorizer fitted on training data.")

    # 3. Transform titles with vectorizer once
    def vectorize_batch(examples):
        import numpy as np
        freq = vectorizer.transform(examples["clean_title"]).toarray().astype(np.float32)
        return {"freq_inputs": freq}

    data = data.map(vectorize_batch, batched=True, num_proc=num_proc)

    # 4. Tokenize with BERT once
    def tokenize_batch(examples):
        tokenized = tokenizer(
            examples["title"],
            padding="max_length",
            truncation=True,
            max_length=max_seq_length
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"]
        }

    data = data.map(tokenize_batch, batched=True, num_proc=num_proc)

    # 5. Convert titles into tokens for W2V
    def split_tokens(examples):
        tokens_list = [t.split() for t in examples["clean_title"]]
        return {"tokens": tokens_list}

    data = data.map(split_tokens, batched=True, num_proc=num_proc)

    # Build an embedding dictionary for all unique tokens (do this once before embedding map)
    unique_tokens = set()
    for tokens in data["tokens"]:
        unique_tokens.update(tokens)

    embedding_dim = w2v_model.vector_size
    embedding_dict = {}
    for tk in unique_tokens:
        if tk in w2v_model:
            embedding_dict[tk] = w2v_model[tk].astype(np.float32)
        else:
            embedding_dict[tk] = np.zeros((embedding_dim,), dtype=np.float32)

    def w2v_embedding_batch(examples):
        import numpy as np
        batch_w2v = []
        for tokens in examples["tokens"]:
            vectors = [embedding_dict[tk] for tk in tokens[:max_seq_length]]
            if len(vectors) < max_seq_length:
                vectors += [np.zeros((embedding_dim,), dtype=np.float32)] * (max_seq_length - len(vectors))
            batch_w2v.append(vectors)
        return {"pos_inputs": batch_w2v}


    data = data.map(w2v_embedding_batch, batched=True, batch_size=32, num_proc=num_proc)

    # 7. Create labels
    def make_labels(examples):
        labels = examples["labels"]
        return {"labels": labels}

    data = data.map(make_labels, batched=True, num_proc=num_proc)

    # Convert freq_inputs and pos_inputs to torch tensors in a final map step
    def to_tensors(examples):
        import torch

        freq_inputs = torch.tensor(examples["freq_inputs"], dtype=torch.float32)
        input_ids = torch.tensor(examples["input_ids"])
        attention_mask = torch.tensor(examples["attention_mask"])
        pos_inputs = torch.tensor(examples["pos_inputs"], dtype=torch.float32)
        labels = torch.tensor(examples["labels"],dtype=torch.long)

        # seq_inputs shape: (batch_size, 2, seq_len)
        seq_inputs = torch.stack([input_ids, attention_mask], dim=1)

        return {
            "freq_inputs": freq_inputs,
            "seq_inputs": seq_inputs,
            "pos_inputs": pos_inputs,
            "labels": labels
        }

    # Apply final conversion to tensor
    processed_data = data.map(to_tensors, batched=True, num_proc=num_proc)

    return processed_data, vectorizer


In [5]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True) #PATH TO PRETRAINED WORD2VEC MODEL

dataset_train,vectorizer = preprocess_data(
    data=dataset_train,
    mode="train",
    w2v_model=w2v_model,
    max_features=8192,
    max_seq_length=128
)

dataset_test, _ = preprocess_data(
    data=dataset_test,
    mode="test",
    vectorizer=vectorizer,
    w2v_model=w2v_model,
    max_features=8192,
    max_seq_length=128
)

vectorizer fitted on training data.


In [15]:
from transformers import AutoModel, AutoConfig
config = AutoConfig.from_pretrained("CISProject/News-Headline-Classifier-Notebook")
model = AutoModel.from_pretrained("CISProject/News-Headline-Classifier-Notebook",config = config)

ValueError: The checkpoint you are trying to load has model type `headlineclassifier` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
# Define a collate function to handle the batched data
def collate_fn(batch):
    freq_inputs = torch.stack([torch.tensor(item["freq_inputs"]) for item in batch])
    seq_inputs = torch.stack([torch.tensor(item["seq_inputs"]) for item in batch])
    pos_inputs = torch.stack([torch.tensor(item["pos_inputs"]) for item in batch])
    labels = torch.tensor([torch.tensor(item["labels"],dtype=torch.long) for item in batch])
    return {"freq_inputs": freq_inputs, "seq_inputs": seq_inputs, "pos_inputs": pos_inputs}, labels

train_loader = DataLoader(dataset_train, batch_size=config.train["batch_size"], shuffle=True,collate_fn=collate_fn)
test_loader = DataLoader(dataset_test, batch_size=config.train["batch_size"], shuffle=False,collate_fn=collate_fn)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
def evaluate_model(model, val_loader, criterion, device="cuda"):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_inputs, labels in tqdm(val_loader, desc="Testing", leave=False):
            freq_inputs = batch_inputs["freq_inputs"].to(device)
            seq_inputs = batch_inputs["seq_inputs"].to(device)
            pos_inputs = batch_inputs["pos_inputs"].to(device)
            labels = labels.to(device)

            preds = model({"freq_inputs": freq_inputs, "seq_inputs": seq_inputs, "pos_inputs": pos_inputs})
            loss = criterion(preds, labels)
            _, preds = torch.max(preds, dim=1)
            # preds = (torch.sigmoid(preds) > 0.5).float()
            val_loss += loss.item()
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return accuracy_score(all_labels, all_preds), classification_report(all_labels, all_preds)


accuracy, report = evaluate_model(model, test_loader, criterion)
print(f"Accuracy: {accuracy:.4f}")
print(report)